In [ ]:
import xarray as xr
from matplotlib import pyplot as plt
import cftime._cftime as cft
import holoviews as hv
#from holoviews import opts
#hv.extension('matplotlib')
#opts.defaults(opts.Scatter3D(color='Value', cmap='fire', edgecolor='black', s=50))

| Experiment Name | Description | Parameters |
| :- | :- | :- |
| EXP_dom   |   No changes in usrdef                                |                                                           |
| EXP_dom2  |   changes in usrdef_hgr                               | do_loop_substitute in usrdef_hgr                          |
| EXP_dom3  |   changes in usrdef_zgr                               | 1, jpi/ 1, jpj -> DO_2D(nn), max/min lambda, max/min phi  |
| EXP_dom4  |   fixed index shift for lam and lon in usrdef_hgr     |  mig0(ii) -> mig0(ii)-1                                   |
| EXP_dom5  |   index shift for lam_max and lon_max in usrdef_zgr   | MAXVAL(...jpi-1) -> MAXVAL(...jpi-2)                      |
| EXP_dom6  |   3D loop in e3u,e3uw,e3vw,e3f...                     | do_loop_substitute in zgr_lib                             |

In [ ]:
path_0 = "/home/dkamm/NEMO4.0/tests/BASIN_compiled_v0.1.0/EXP_dom/"
path_2 = "/home/dkamm/NEMOGCM/tests/BASIN2/"

In [ ]:
def clean_coords(xrdataset, start_year):
    try:
        shift = cft.timedelta(days=360 * start_year)
        datetimeindex = (xrdataset.indexes['time_counter'] + shift).to_datetimeindex
    except:
        datetimeindex = xrdataset.indexes['time_counter']
    try:
        latitude  = xrdataset.nav_lat[:,0].values
        longitude = xrdataset.nav_lon[0,:].values
    except:
        latitude  = xrdataset.nav_lat_grid_T[:,0].values
        longitude = xrdataset.nav_lon_grid_T[0,:].values
    try:
        xrdataset = xrdataset.rename_dims({'x_grid_T': 'x' ,'y_grid_T': 'y'})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3t.cumsum('deptht')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3u.cumsum('depthu')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3v.cumsum('depthv')})
    except:
        pass
    try:
        xrdataset = xrdataset.assign_coords({'z': 5. - xrdataset.e3w.cumsum('depthw')})
    except:
        pass
    return xrdataset.assign_coords(dict({'time_counter':datetimeindex, 'y': latitude, 'x': longitude}))

In [ ]:
dom_0 = xr.load_dataset(path_0 + 'domain_cfg_out.nc')
dom_0 = clean_coords(dom_0, 2000)

In [ ]:
dom_0.gdepw_0.isel(nav_lev=-1).plot()

In [ ]:
dom_0.e3t_0.isel(y=10).sum('nav_lev').plot()

In [ ]:
dom_8 = xr.load_dataset(path_2 + 'EXP_dom8/domain_cfg_out.nc')
dom_8 = clean_coords(dom_8, 2000)

dom_7 = xr.load_dataset(path_2 + 'EXP_dom7/domain_cfg_out.nc')
dom_7 = clean_coords(dom_7, 2000)

dom_6 = xr.load_dataset(path_2 + 'EXP_dom6/domain_cfg_out.nc')
dom_6 = clean_coords(dom_6, 2000)

dom_5 = xr.load_dataset(path_2 + 'EXP_dom5/domain_cfg_out.nc')
dom_5 = clean_coords(dom_5, 2000)

dom_4 = xr.load_dataset(path_2 + 'EXP_dom4/domain_cfg_out.nc')
dom_4 = clean_coords(dom_4, 2000)

dom_3 = xr.load_dataset(path_2 + 'EXP_dom3/domain_cfg_out.nc')
dom_3 = clean_coords(dom_3, 2000)

dom_2 = xr.load_dataset(path_2 + 'EXP_dom2/domain_cfg_out.nc')
dom_2 = clean_coords(dom_2, 2000)

dom_ref = xr.load_dataset(path_2 + 'EXP_dom/domain_cfg_out.nc')
dom_ref = clean_coords(dom_ref, 2000)

In [ ]:
dom_dbg = xr.load_dataset('/home/dkamm/NEMOGCM/tests/BASIN_debug/EXP01/domain_cfg_out.nc')
dom_dbg = clean_coords(dom_dbg, 2000)

In [ ]:
dom_7.y - 0.94

In [ ]:
dom_8

In [ ]:
dom_8.e2t - dom_8.e1t

In [ ]:
dom_.isel(nav_lev=20, x=-1, y=[-3, -2, -1])

In [ ]:
dom_7.isel(nav_lev=20, x=-1, y=[-4, -3, -2, -1])

In [ ]:
slope_0 = xr.load_dataset(path_0 + 'mesh_mask.nc')
slope_0 = clean_coords(slope_0, 2000)

slope_2 = xr.load_dataset(path_2 + 'EXP_dom7/mesh_mask.nc')
slope_2 = clean_coords(slope_2, 2000)

In [ ]:
slope_0.mbathy

In [ ]:
slope_2.ratio_slopej.plot()

In [ ]:
(dom_ref - dom_0).max()

In [ ]:
(dom_0.e3w_0.cumsum('nav_lev') - dom_4.e3w_0.cumsum('nav_lev'))

In [ ]:
dom_0.e2f

In [ ]:
hv_ds = hv.Dataset(dom_0.e3w_0.cumsum('nav_lev') - dom_5.e3w_0.cumsum('nav_lev'))

In [ ]:
domain = hv_ds.to(hv.QuadMesh, kdims=["x", "y"], dynamic=False)
domain.opts(colorbar=True, fig_size=200, cmap='RdBu_r')